In [1]:
try:
    from google.colab import drive
    drive.mount("/content/drive")
except:
    raise AssertionError()

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/DeepLearning/KoBart/KoBART-summarization

/content/drive/MyDrive/DeepLearning/KoBart/KoBART-summarization


In [ ]:
# Mecab 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# mecab-python의 버전 오류로 인해 아래 패키지를 설치하면 코랩에서 Mecab을 사용가능
!pip install mecab-python3

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
openjdk-8-jdk is already the newest version (8u392-ga-1~22.04).
0 upgraded, 0 newly

In [ ]:
# 정상 동작 확인
from konlpy.tag import Mecab

mecab = Mecab()

In [ ]:
mecab.morphs('한글 형태소 분석기(미켑)로 테스트를 해보았습니다. 정상 설치 및 동작이 잘 됩니다.')

['한글',
 '형태소',
 '분석기',
 '(',
 '미',
 '켑',
 ')',
 '로',
 '테스트',
 '를',
 '해',
 '보',
 '았',
 '습니다',
 '.',
 '정상',
 '설치',
 '및',
 '동작',
 '이',
 '잘',
 '됩니다',
 '.']

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration
!python get_model_binary.py --model_binary '/content/drive/MyDrive/DeepLearning/KoBart/KoBART-summarization/checkpoint/model_chp/epoch=06-val_loss=0.384.ckpt'

# 모델 바이너리 파일 경로
model_binary_path = '/content/drive/MyDrive/DeepLearning/KoBart/KoBART-summarization/kobart_summary'

# KoBART 모델 및 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')
model = BartForConditionalGeneration.from_pretrained(model_binary_path)

Traceback (most recent call last):
  File "/content/drive/MyDrive/DeepLearning/KoBart/KoBART-summarization/get_model_binary.py", line 2, in <module>
    from train import KoBARTConditionalGeneration
  File "/content/drive/MyDrive/DeepLearning/KoBart/KoBART-summarization/train.py", line 5, in <module>
    from loguru import logger
ModuleNotFoundError: No module named 'loguru'


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
import pandas as pd

# 테스트 데이터셋 로드
test_data_path = '/content/drive/MyDrive/DeepLearning/KoBart/KoBART-summarization/data/test_100.tsv'
test_df = pd.read_csv(test_data_path, delimiter='\t')

In [ ]:
!pip install rouge

In [ ]:
from konlpy.tag import Mecab  # konlpy에서 Mecab 모듈 가져오기
from rouge import Rouge

# Mecab 초기화
mecab = Mecab()  # Mecab()을 사용하여 초기화

# 정확도 누적 변수 초기화
total_accuracy = {'rouge-1': 0.0, 'rouge-2': 0.0, 'rouge-l': 0.0}
results = []

def calculate_rouge(reference, generated_tokens):
    # 레퍼런스 텍스트를 mecab을 사용하여 토큰화
    reference_tokens = mecab.morphs(reference)

    # 토큰 목록을 문자열로 변환
    generated_summary = ' '.join(generated_tokens)
    reference_summary = ' '.join(reference_tokens)

    rouge = Rouge()
    scores = rouge.get_scores(generated_summary, reference_summary)

     # 'rouge-1'에 대한 F1 점수
    rouge_1_score = scores[0]['rouge-1']['f']
    # 'rouge-2'에 대한 F1 점수
    rouge_2_score = scores[0]['rouge-2']['f']
    # 'rouge-l'에 대한 F1 점수
    rouge_l_score = scores[0]['rouge-l']['f']

    return {'rouge-1': rouge_1_score, 'rouge-2': rouge_2_score, 'rouge-l': rouge_l_score}

# 테스트 데이터셋 순회
for index, row in test_df.iterrows():
    # 입력 텍스트를 토큰화하여 인코딩
    input_ids = tokenizer.encode(row['news'], return_tensors="pt", max_length=1024, truncation=True)

    # 모델을 사용하여 요약 생성
    summary_ids = model.generate(input_ids, max_length=110, min_length=20, length_penalty=2.0, num_beams=4, early_stopping=True)

    # 요약 결과 디코딩
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # 모델이 생성한 요약 텍스트를 mecab을 사용하여 토큰화
    generated_tokens = mecab.morphs(generated_summary)

    # 정확도 계산 및 누적
    # accuracy = calculate_accuracy(row['summary'], generated_summary)
    # 정확도 계산을 ROUGE 스코어로 대체
    accuracy = calculate_rouge(row['summary'], generated_tokens)
    total_accuracy['rouge-1'] += accuracy['rouge-1']
    total_accuracy['rouge-2'] += accuracy['rouge-2']
    total_accuracy['rouge-l'] += accuracy['rouge-l']

    # 결과 저장
    results.append({
        'index': index,
        'actual_summary': row['summary'],
        'generated_summary': generated_summary,
        'generated_tokens': generated_tokens,  # 토큰화된 결과 저장
        'accuracy': accuracy
    })

    # 결과 출력
    print(f"Index: {index}")
    print(f"Actual Summary: {row['summary']}")
    print(f"Generated Summary: {generated_summary}")
    print(f"Accuracy (ROUGE-1): {accuracy['rouge-1']}")
    print(f"Accuracy (ROUGE-2): {accuracy['rouge-2']}")
    print(f"Accuracy (ROUGE-l): {accuracy['rouge-l']}")
    print("="*50)

# 평균 정확도 계산
average_accuracy = {
    'rouge-1': total_accuracy['rouge-1'] / len(test_df),
    'rouge-2': total_accuracy['rouge-2'] / len(test_df),
    'rouge-l': total_accuracy['rouge-l'] / len(test_df)
}

# 평균 정확도 출력
print("평균 ROUGE-1 스코어:", average_accuracy['rouge-1'])
print("평균 ROUGE-2 스코어:", average_accuracy['rouge-2'])
print("평균 ROUGE-l 스코어:", average_accuracy['rouge-l'])

Index: 0
Actual Summary: 태왕의 '성당 태왕아너스 메트로'모델하우스는 초역세권 입지와 변화하는 라이프스타일에 맞춘 혁신평면으로 오픈 당일부터 관람객의 줄이 이어지면서 관람객의 호평을 받았다.
Generated Summary: 태왕의 '성당 태왕아너스 메트로' 모델하우스를 찾은 방문객들은 초역세권 입지와 변화하는 라이프스타일에 맞춘 혁신평면에 호평을 보내며 초역세권 입지와 변화하는 라이프스타일에 맞춘 혁신평면이 아파트 선택에 미치는 영향력이 커지고 있다.
Accuracy (ROUGE-1): 0.6923076873208417
Accuracy (ROUGE-2): 0.5393258377275597
Accuracy (ROUGE-l): 0.666666661679816
Index: 1
Actual Summary: 최근 늘어나고 있는 집회시위에서 소음이 문제가 되고 있는데 현행 집회와 시위에 관한 법률에서는 소음을 규제하고 있으나 그 기준이 일반 시민이 느끼는 기준과 괴리감이 심해 집회인들은 자신들의  요구를 전달하기 위해 확성기 사용은 불가피하다고 주장하고 있고 대다수 주민들은 소음으로 인해 많은 불편함을 느끼고 있다.
Generated Summary: 최근 늘어나고 있는 집회시위에서 소음이 문제가 되고 있는데 현행 집회와 시위에 관한 법률에서는 확성기 등에 의한 사용 제한으로 소음을 규제하고 있으나 그 기준이 일반 시민이 느끼는 기준과 매우 격차가 심해 대다수 주민들이 많은 고통과 함께 민원을 제기하고 있다.
Accuracy (ROUGE-1): 0.757281548421152
Accuracy (ROUGE-2): 0.5874125824676024
Accuracy (ROUGE-l): 0.6990291212366859
Index: 2
Actual Summary: 지난 19일 고흥군에 따르면 송귀근 군수와 주민 등 300여 명이 참석한 가운데 귀농·귀촌인의 안정적인 정착을 돕고 영농교육 등을 담당하기 위해 지난해부터 추진했던 ‘귀농귀촌 행복학교'의 개교 행사